## Dow Jones 5-Stock Portfolio 

In [68]:
!pip install yfinance


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [69]:
# Import the yfinance library to access Yahoo Finance data.
import yfinance as yf
import pandas as pd
import numpy as np

import requests
import zipfile
import io

from sklearn.linear_model import Lasso

The following code uses the Britten-Jones (1999, JF) methodology to construct a portfolio using yfinance price data.

In [72]:
# Dow Jones tickers
dow_tickers = [
    'AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT', 'DOW'
]

# Download adjusted closing prices
# data = yf.download(dow_tickers, period='10y', interval='1mo')['Close']

# If the data download doesn't work, import the data manually
data = pd.read_csv('data.csv', index_col=0)
data = data.dropna(axis=1)

# Compute daily log returns
log_returns = np.log(data / data.shift(1)).dropna()

log_returns

,AAPL,AMGN,AXP,BA,CAT,CVX,DIS,HD,HON,IBM,...,MCD,MMM,MRK,NKE,PG,TRV,UNH,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,
1985-07-01,-0.125617,-0.098439,-0.064221,0.075304,0.107245,-0.016864,-0.074621,-0.009478,0.045853,0.059792,...,-0.040975,0.025318,0.009661,0.045458,0.019846,-0.012142,0.032787,-0.043719,-0.114959,-0.097209
1985-08-01,-0.056706,0.050428,-0.070204,-0.002594,-0.020583,0.023530,0.025497,-0.100084,-0.072601,-0.036826,...,-0.013974,-0.044736,0.011014,0.105365,0.015652,0.001744,-0.101782,0.029229,0.039806,0.044453
1985-09-01,0.048801,0.048012,0.006042,-0.053357,-0.017514,-0.000458,-0.063513,-0.065241,0.066982,-0.013410,...,0.003861,0.004891,-0.045870,0.067655,-0.019759,-0.057363,-0.303680,-0.057885,-0.098353,-0.079066
1985-10-01,0.167654,0.271932,0.100196,0.010959,0.003527,0.055844,0.095872,-0.011297,-0.011332,0.047300,...,0.028492,0.022765,0.073448,0.009305,0.117044,0.079759,0.216668,0.074154,0.072950,0.104312
1985-11-01,0.077465,0.057820,0.071553,0.068448,0.119625,-0.022618,0.073330,0.000000,0.052717,0.073283,...,0.116766,0.081772,0.109643,-0.018696,0.068930,0.051464,0.038222,0.090673,0.100322,0.111475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01,-0.059308,0.090822,0.067296,-0.002716,0.023646,0.029591,0.019662,0.057421,-0.009653,0.151150,...,0.001884,0.164686,0.001316,0.021215,-0.009951,0.021829,0.073738,-0.015118,0.096965,0.085121
2025-02-01,0.024444,0.076333,-0.050995,-0.010765,-0.073232,0.061291,0.006524,-0.038047,-0.049620,-0.012831,...,0.065783,0.019004,-0.068798,0.032370,0.052287,0.052864,-0.132939,0.107576,0.038173,0.004574
2025-03-01,-0.083915,0.019312,-0.112080,-0.023641,-0.042010,0.064248,-0.142358,-0.078957,-0.000032,-0.008447,...,0.013018,-0.049799,-0.027363,-0.224151,-0.019869,0.022833,0.097784,0.051108,0.044859,-0.116225


In [73]:
# Dependent variable
y = np.ones(log_returns.shape[0])

# Finding the Lasso lambda (alpha) that selects exactly 5 stocks
alphas = np.linspace(0.000001, 0.01, 1000)
selected_alpha = None
selected_coefs = None

for alpha in alphas:
    lasso = Lasso(alpha=alpha, fit_intercept=False, positive=True, max_iter=10000)
    lasso.fit(log_returns, y)
    if np.sum(lasso.coef_ != 0) == 5:
        selected_alpha = alpha
        selected_coefs = lasso.coef_
        break

# Display results
optimal_portfolio = pd.Series(selected_coefs, index=log_returns.columns)
optimal_portfolio = optimal_portfolio[optimal_portfolio != 0]

print("Optimal Portfolio Stocks and their weights:")
print(optimal_portfolio)
print(f"\nChosen Lasso alpha (lambda): {selected_alpha}")

Optimal Portfolio Stocks and their weights:
AAPL    0.279596
AMGN    0.546323
HD      0.893697
PG      0.045482
UNH     0.346883
dtype: float64

Chosen Lasso alpha (lambda): 0.0071874684684684696


In [74]:
for i in range(0, len(optimal_portfolio)):
    print(f"{i}: {optimal_portfolio[i]/sum(optimal_portfolio)}")

0: 0.1323857330964561
1: 0.25867803703111975
2: 0.4231558662102086
3: 0.02153521847015985
4: 0.16424514519205577


Calculating alphas

In [75]:
log_returns.reset_index(inplace=True)
log_returns.rename(columns={"Date": "date"}, inplace=True)
log_returns["date"] = pd.to_datetime(log_returns["date"])
log_returns

,date,AAPL,AMGN,AXP,BA,CAT,CVX,DIS,HD,HON,...,MCD,MMM,MRK,NKE,PG,TRV,UNH,VZ,WBA,WMT
0,1985-07-01,-0.125617,-0.098439,-0.064221,0.075304,0.107245,-0.016864,-0.074621,-0.009478,0.045853,...,-0.040975,0.025318,0.009661,0.045458,0.019846,-0.012142,0.032787,-0.043719,-0.114959,-0.097209
1,1985-08-01,-0.056706,0.050428,-0.070204,-0.002594,-0.020583,0.023530,0.025497,-0.100084,-0.072601,...,-0.013974,-0.044736,0.011014,0.105365,0.015652,0.001744,-0.101782,0.029229,0.039806,0.044453
2,1985-09-01,0.048801,0.048012,0.006042,-0.053357,-0.017514,-0.000458,-0.063513,-0.065241,0.066982,...,0.003861,0.004891,-0.045870,0.067655,-0.019759,-0.057363,-0.303680,-0.057885,-0.098353,-0.079066
3,1985-10-01,0.167654,0.271932,0.100196,0.010959,0.003527,0.055844,0.095872,-0.011297,-0.011332,...,0.028492,0.022765,0.073448,0.009305,0.117044,0.079759,0.216668,0.074154,0.072950,0.104312
4,1985-11-01,0.077465,0.057820,0.071553,0.068448,0.119625,-0.022618,0.073330,0.000000,0.052717,...,0.116766,0.081772,0.109643,-0.018696,0.068930,0.051464,0.038222,0.090673,0.100322,0.111475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2025-01-01,-0.059308,0.090822,0.067296,-0.002716,0.023646,0.029591,0.019662,0.057421,-0.009653,...,0.001884,0.164686,0.001316,0.021215,-0.009951,0.021829,0.073738,-0.015118,0.096965,0.085121
475,2025-02-01,0.024444,0.076333,-0.050995,-0.010765,-0.073232,0.061291,0.006524,-0.038047,-0.049620,...,0.065783,0.019004,-0.068798,0.032370,0.052287,0.052864,-0.132939,0.107576,0.038173,0.004574
476,2025-03-01,-0.083915,0.019312,-0.112080,-0.023641,-0.042010,0.064248,-0.142358,-0.078957,-0.000032,...,0.013018,-0.049799,-0.027363,-0.224151,-0.019869,0.022833,0.097784,0.051108,0.044859,-0.116225
477,2025-04-01,-0.044321,-0.068512,-0.009861,0.071768,-0.064272,-0.206633,-0.081775,-0.010181,-0.005921,...,0.028795,-0.055653,-0.043536,-0.113207,-0.047157,0.002834,-0.237087,-0.029078,-0.018067,0.105080


In [76]:
# URL of the ZIP file
url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_CSV.zip"

# Download the ZIP file
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))

# Extract the CSV file (assuming it is the first file in the ZIP)
csv_filename = z.namelist()[0]
with z.open(csv_filename) as file:
    df_rf = pd.read_csv(file, skiprows=3, index_col=0)
    
# split the df before annual data begins
df_rf = df_rf.loc[:"202503"]

df_rf.index = pd.to_datetime(df_rf.index, format="%Y%m")
df_rf = df_rf.apply(pd.to_numeric)
df_rf = df_rf/100
df_rf.reset_index(inplace=True)
df_rf.rename(columns={'index': 'date'}, inplace=True)

# Display the first few rows
print(df_rf.head())

        date  Mkt-RF     SMB     HML     RMW     CMA      RF
0 1963-07-01 -0.0039 -0.0048 -0.0081  0.0064 -0.0115  0.0027
1 1963-08-01  0.0508 -0.0080  0.0170  0.0040 -0.0038  0.0025
2 1963-09-01 -0.0157 -0.0043  0.0000 -0.0078  0.0015  0.0027
3 1963-10-01  0.0254 -0.0134 -0.0004  0.0279 -0.0225  0.0029
4 1963-11-01 -0.0086 -0.0085  0.0173 -0.0043  0.0227  0.0027


In [77]:
df=pd.merge(log_returns,df_rf, how='inner', on='date')
print(df.head())

        date      AAPL      AMGN       AXP        BA       CAT       CVX  \
0 1985-07-01 -0.125617 -0.098439 -0.064221  0.075304  0.107245 -0.016864   
1 1985-08-01 -0.056706  0.050428 -0.070204 -0.002594 -0.020583  0.023530   
2 1985-09-01  0.048801  0.048012  0.006042 -0.053357 -0.017514 -0.000458   
3 1985-10-01  0.167654  0.271932  0.100196  0.010959  0.003527  0.055844   
4 1985-11-01  0.077465  0.057820  0.071553  0.068448  0.119625 -0.022618   

        DIS        HD       HON  ...       UNH        VZ       WBA       WMT  \
0 -0.074621 -0.009478  0.045853  ...  0.032787 -0.043719 -0.114959 -0.097209   
1  0.025497 -0.100084 -0.072601  ... -0.101782  0.029229  0.039806  0.044453   
2 -0.063513 -0.065241  0.066982  ... -0.303680 -0.057885 -0.098353 -0.079066   
3  0.095872 -0.011297 -0.011332  ...  0.216668  0.074154  0.072950  0.104312   
4  0.073330  0.000000  0.052717  ...  0.038222  0.090673  0.100322  0.111475   

   Mkt-RF     SMB     HML     RMW     CMA      RF  
0 -0.0072 

In [78]:
excluded = ['date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
ticker_columns = [col for col in df.columns if col not in excluded]

stocks_excess = df[ticker_columns].subtract(df.RF, axis='index')
stocks_excess

,AAPL,AMGN,AXP,BA,CAT,CVX,DIS,HD,HON,IBM,...,MCD,MMM,MRK,NKE,PG,TRV,UNH,VZ,WBA,WMT
0,-0.131817,-0.104639,-0.070421,0.069104,0.101045,-0.023064,-0.080821,-0.015678,0.039653,0.053592,...,-0.047175,0.019118,0.003461,0.039258,0.013646,-0.018342,0.026587,-0.049919,-0.121159,-0.103409
1,-0.062206,0.044928,-0.075704,-0.008094,-0.026083,0.018030,0.019997,-0.105584,-0.078101,-0.042326,...,-0.019474,-0.050236,0.005514,0.099865,0.010152,-0.003756,-0.107282,0.023729,0.034306,0.038953
2,0.042801,0.042012,0.000042,-0.059357,-0.023514,-0.006458,-0.069513,-0.071241,0.060982,-0.019410,...,-0.002139,-0.001109,-0.051870,0.061655,-0.025759,-0.063363,-0.309680,-0.063885,-0.104353,-0.085066
3,0.161154,0.265432,0.093696,0.004459,-0.002973,0.049344,0.089372,-0.017797,-0.017832,0.040800,...,0.021992,0.016265,0.066948,0.002805,0.110544,0.073259,0.210168,0.067654,0.066450,0.097812
4,0.071365,0.051720,0.065453,0.062348,0.113525,-0.028718,0.067230,-0.006100,0.046617,0.067183,...,0.110666,0.075672,0.103543,-0.024796,0.062830,0.045364,0.032122,0.084573,0.094222,0.105375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,0.045315,-0.127833,0.119154,0.036235,0.076089,0.080427,0.195754,0.082044,0.120417,0.091389,...,0.009263,0.034631,-0.010668,0.017040,0.083664,0.074559,0.073846,0.062617,-0.051628,0.117099
473,0.051088,-0.077582,-0.029937,0.126190,-0.116583,-0.105085,-0.057205,-0.096654,-0.029479,-0.029750,...,-0.024592,-0.032212,-0.025180,-0.043850,-0.070659,-0.103006,-0.191227,-0.106958,0.060015,-0.027218
474,-0.063008,0.087122,0.063596,-0.006416,0.019946,0.025891,0.015962,0.053721,-0.013353,0.147450,...,-0.001816,0.160986,-0.002384,0.017515,-0.013651,0.018129,0.070038,-0.018818,0.093265,0.081421
475,0.021144,0.073033,-0.054295,-0.014065,-0.076532,0.057991,0.003224,-0.041347,-0.052920,-0.016131,...,0.062483,0.015704,-0.072098,0.029070,0.048987,0.049564,-0.136239,0.104276,0.034873,0.001274


In [79]:
import statsmodels.api as sm

alphas = {}

for stock in stocks_excess.columns:
    y = stocks_excess[stock]
    X = df[["Mkt-RF", "SMB", "HML"]]
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()
    alphas[stock] = model.params["const"]  # This is the FF3 alpha
    
alpha_df = pd.DataFrame.from_dict(alphas, orient='index', columns=['FF3_alpha'])

In [80]:
alpha_df

,FF3_alpha
AAPL,0.007111
AMGN,0.009246
AXP,-0.004443
BA,-0.004273
CAT,-0.001889
CVX,-0.000285
DIS,-0.002737
HD,0.005424
HON,-0.003405
IBM,-0.002497
